### Load the recorded domalys data with pandas and excel file saved

In [7]:
import pandas as pd

In [12]:
#path to the excel file : 
path = "C:/Users/mcouv/Work/machine-learning/ia_fall_confirmation"
records = pd.read_excel(path + "/record_data_thermal_all.xlsx")

In [13]:
records.head()

,Unnamed: 0,test_id,timestamp,thermal_top,thermal_bot,fall,temperature
0,0,7.4,16:22:55,[[1984 1901 1873 1880 1957 1901 1926 1982 1902...,[[1796 1829 1907 1895 1879 1848 1889 1909 1928...,1,20.53
1,1,7.4,16:22:55,[[1904 1872 1881 1887 1910 1907 1908 1897 1908...,[[1561 1703 1787 1777 1702 1820 1749 1847 1893...,1,20.53
2,2,7.4,16:22:55,[[1892 1863 1907 1850 1903 1950 1973 1914 1948...,[[2053 1829 1907 1861 1937 1934 1915 1934 1928...,1,20.53
3,3,7.4,16:22:55,[[1938 1939 1907 1938 1903 1901 1926 1937 1972...,[[1628 1691 1816 1734 1844 1842 1795 1788 1809...,1,20.53
4,4,7.4,16:22:55,[[1904 1910 1915 1916 1883 1857 1884 1942 1931...,[[1868 1776 1885 1794 1777 1834 1815 1834 1880...,1,20.53


##### Pour chaques frame du dataset, il faut reconstituer chaque enregistrement en concatenent les deux thermal top et thermal bot

In [14]:
import numpy as np
import cv2

def generate_image_from_dataset(record, size=(640, 320)):
    try:
        string_listtop = record["thermal_top"].replace("\n", "").replace("  ", " ").replace(" ",",").replace("[[","").replace("]]","").split("],[")
        string_listbot = record["thermal_bot"].replace("\n", "").replace("  ", " ").replace(" ",",").replace("[[","").replace("]]","").split("],[")
    except Exception as e:
        print("Error while parsing the record")
        print(e)
        return None

    np_listtop = np.array([np.array([int(x) for x in string.split(",")]) for string in string_listtop])
    np_listbot = np.array([np.array([int(x) for x in string.split(",")]) for string in string_listbot])
    np_image = np.concatenate((np_listtop, np_listbot), axis=0)

    # Normalisation
    min_val = np.min(np_image)
    max_val = np.max(np_image)

    try:
        np_image_normalized = ((np_image - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    except Exception as e:
        print("Error while normalizing the image")
        print(e)
        return None
    
    # Redimensionner l'image 
    resized_image = cv2.resize(np_image_normalized, size, interpolation=cv2.INTER_NEAREST).astype('uint8')
    return resized_image


In [15]:
import numpy as np
import cv2
import torch

def generate_tensor_from_dataset(record):
    string_listtop = record["thermal_top"].replace("\n", "").replace("  ", " ").replace(" ",",").replace("[[","").replace("]]","").split("],[")
    string_listbot = record["thermal_bot"].replace("\n", "").replace("  ", " ").replace(" ",",").replace("[[","").replace("]]","").split("],[")

    np_listtop = np.array([np.array([int(x) for x in string.split(",")]) for string in string_listtop])
    np_listbot = np.array([np.array([int(x) for x in string.split(",")]) for string in string_listbot])
    np_image = np.concatenate((np_listtop, np_listbot), axis=0)
    frame_tensor = torch.tensor(np_image)
    return frame_tensor


In [42]:
pip install ffmpeg-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
ffmpeg -i classes/fall/prout.mp4 -vcodec libx264 classes/fall/prout.mp4

SyntaxError: invalid syntax (1621679558.py, line 1)

In [16]:
import time
import os

def convert_videos():
    # Attendre pour s'assurer que la vidéo est entièrement écriture
    command = f"wsl bash ./convert_videos.sh"
    # Exécuter la commande avec subprocess
    res = os.system(command)
    # Vous pouvez également vérifier si la commande s'est terminée avec succès
    if res == 0:
        print(f"Conversion completed successfully!")
    else:
        print(f"Error while converting the videos")


In [ ]:
import cv2
import os
import uuid


def save_video(images_stack, isfall):
    unique_id = str(uuid.uuid4()).replace("-", "")
    # Create and save video
    save_path = "./classes/fall" if isfall == 1 else "./classes/no_fall"
    os.makedirs(save_path, exist_ok=True)
    full_path = f"{save_path}/{unique_id}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') 
    out = cv2.VideoWriter(full_path, fourcc, 10.0, (640, 320), isColor=False)
    for image in images_stack:
        out.write(image)
    out.release()

def generate_videos_from_dataset(records):
    current = None
    count_test = 0
    count_frame = 0
    video_images = []

    for record in records.iterrows():
    
        #first test we setup the current test and save the first image
        if current is None:
            current = record[1]['test_id']
            isfall = record[1]['fall']
            print(f"new test : {current}")
            count_test = 1
            try :
                resized_image = generate_image_from_dataset(record[1])
                video_images.append(resized_image)
                count_frame+=1
                continue
            except Exception as e:
                print("Error while generating the image")
                print(e)
                continue
            
        #If test is different we save the video and reset the counter
        if record[1]['test_id'] != current:
            #save the last test
            print(f"Test ended with {count_frame} frames")
            save_video(video_images, isfall)
            print(f"new test : {current}")
            video_images = []
            current = record[1]['test_id']
            count_test += 1
            count_frame = 0
            isfall = record[1]['fall']

        #save the image
        try :
            resized_image = generate_image_from_dataset(record[1])
            video_images.append(resized_image)
            count_frame+=1
            continue
        except Exception as e:
            print("Error while generating the image")
            print(e)
            continue
        if count_test == 10:
            break

    #save the last test
    print(f"Test ended with {count_frame} frames")
    save_video(video_images, isfall)
    convert_videos()
    

    

In [11]:
import os
import torch
from torchvision.io import read_video
from torch.utils.data import Dataset

import cv2

def load_video_with_opencv(video_path):
    vid = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = vid.read()
        if not ret:
            break
        # Convertir en niveaux de gris (même si c'est déjà le cas)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(torch.tensor(gray_frame))
    vid.release()
    video_tensor = torch.stack(frames)
    return video_tensor.unsqueeze(1)  # Ajoute un canal


class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
        - root_dir (string): Dossier avec toutes les vidéos.
        - transform (callable, optional): Transformation optionnelle à appliquer
            sur une vidéo.
        """
        self.root_dir = root_dir
        self.transform = transform

        self.classes = os.listdir(root_dir)
        self.classes.sort()
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        self.video_list = []
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            for video_name in os.listdir(cls_dir):
                self.video_list.append((os.path.join(cls_dir, video_name), self.class_to_idx[cls_name]))

    def __len__(self):
        return len(self.video_list)

    def __getitem__(self, idx):
        video_path, label = self.video_list[idx]
        
        # Charger la vidéo. Pour simplifier, nous chargerons seulement les images ici.
        # Pour un cas d'utilisation réel, vous pourriez vouloir travailler avec 
        # les images et l'audio.
        video = load_video_with_opencv(video_path)
        
        if self.transform:
            video = self.transform(video)

        return video, label
    



In [14]:
from torch.utils.data import DataLoader
import cv2
from torch.utils.data import random_split
import random
from torch.utils.data import Subset
import numpy as np



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def uniform_frame_sampling(video_tensor, target_frames=90):
    """
    Sélectionne un nombre uniformément distribué de frames pour que toutes les vidéos aient la même taille.

    Args:
        video_tensor (torch.Tensor): La vidéo originale de forme (T, C, H, W)
        target_frames (int): Nombre de frames cibles à obtenir

    Returns:
        torch.Tensor: Vidéo échantillonnée de forme (target_frames, C, H, W)
    """

    # Obtenir le nombre actuel de frames
    current_frames = video_tensor.shape[0]

    # Si la vidéo actuelle a exactement le nombre cible de frames, la renvoyer telle quelle
    if current_frames == target_frames:
        return video_tensor

    # Calculer les indices des frames à échantillonner
    indices = torch.linspace(0, current_frames - 1, target_frames).long()

    return video_tensor[indices]

# Pour l'utiliser dans le VideoDataset :
def video_resize_and_sample(video_tensor, size=(16, 8), T=90):
    # Resize
    resized_video = []
    for frame in video_tensor:
        # S'assurer que la frame est en niveaux de gris (HxW) et non (1xHxW)
        frame = frame.squeeze(0)
        frame = frame.numpy()
        resized_frame = cv2.resize(frame, size, interpolation=cv2.INTER_NEAREST)
        resized_video.append(torch.tensor(resized_frame).unsqueeze(0))  # Ajouter de nouveau le canal
    
    resized_video_tensor = torch.stack(resized_video)
    # Uniform sampling
    sampled_video_tensor = uniform_frame_sampling(resized_video_tensor, T)
     # Normalisation entre [-1, 1]
    normalized_tensor = (sampled_video_tensor / 127.5)

    return normalized_tensor.to(device)

# Utilisation
transform = video_resize_and_sample
video_dataset = VideoDataset(root_dir='./classes/', transform=transform)


train_indices = []
test_indices = []

# Pour chaque classe, récupérez les indices de ses vidéos.
for cls_name in video_dataset.classes:
    cls_indices = [i for i, (path, label) in enumerate(video_dataset.video_list) if label == video_dataset.class_to_idx[cls_name]]
    
    # Mélangez ces indices.
    random.shuffle(cls_indices)
    
    # Séparez-les en fonction des ratios d'entraînement et de test.
    cls_train_size = int(0.8 * len(cls_indices))
    train_indices.extend(cls_indices[:cls_train_size])
    test_indices.extend(cls_indices[cls_train_size:])

# Créez des sous-ensembles d'entraînement et de test en utilisant ces indices.
train_dataset = Subset(video_dataset, train_indices)
test_dataset = Subset(video_dataset, test_indices)

# DataLoader pour chaque ensemble:
train_loader = DataLoader(train_dataset, batch_size=20, num_workers=1,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Test
# count = 0
# for video_batch, labels in train_loader:
#     video_batch, labels = video_batch.to(device), labels.to(device)
#     print(f"batch shape = {video_batch.shape}, device = {video_batch.device}")
#     print(f"labels shape = {labels.shape}, device = {labels.device}")
#     count += 1
#     if count == 1:
#         image = video_batch[0][89]
#         print(image.shape)
#         print(image)
#         break

In [15]:
pip install pytorchvideo


     ---------------------------------------- 0.0/132.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/132.7 kB ? eta -:--:--
     ----------- ------------------------- 41.0/132.7 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 132.7/132.7 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/50.2 kB ? eta -:--:--
     ---------------------------------------- 50.2/50.2 kB ? eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
 

In [13]:
import torch.nn as nn


class VideoClassifier(nn.Module):
    def __init__(self):
        super(VideoClassifier, self).__init__()
        
        # La couche d'entrée attend des vidéos de la forme (C, T, H, W) = (1, 90, 8, 16)
        self.conv_layer1 = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=(3, 3, 3), padding=1),  # 3D convolution
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2))
        )
        
        self.conv_layer2 = nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2))
        )
        
        # Ici, on aplatit la sortie pour la connecter à des couches entièrement connectées
        self.fc1 = nn.Linear(64 * 22 * 2 * 4, 512)  # La taille dépend de la sortie du dernier MaxPool3d
        self.fc2 = nn.Linear(512, 1)  # 1 neurone en sortie pour une classification binaire
        
    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = x.view(x.size(0), -1)  # flatten
        x = nn.ReLU()(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()  # la sortie est de forme (batch_size)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du device {device}")

model = VideoClassifier()
model = model.to(device)
model.train()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

num_epochs = 10  # ou tout autre nombre que vous jugez approprié


for epoch in range(num_epochs):
    for video_batch, labels in train_loader:
        video_batch, labels = video_batch.to(device), labels.to(device)
        # Permuter les dimensions T et C
        video_batch = video_batch.permute(0, 2, 1, 3, 4)
        labels = labels.float()  # Convertir les labels en float pour la BCEWithLogitsLoss
        
        # Forward
        outputs = model(video_batch)
        loss = criterion(outputs, labels)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")




Utilisation du device cuda
Epoch 1/10, Loss: 0.6931472420692444


KeyboardInterrupt: 

In [ ]:
pip install pytorch_lightning

array([[1984, 1901, 1873, 1880, 1957, 1901, 1926, 1982, 1902, 1977, 1938,
        2003, 1970, 1982, 1950, 2087],
       [1918, 1888, 1925, 1922, 1888, 1929, 1956, 1998, 1989, 1969, 1935,
        1958, 1952, 1966, 2003, 1921],
       [1989, 1887, 1903, 1951, 1964, 1928, 1941, 1944, 1967, 1939, 1993,
        2383, 2157, 2014, 2036, 1961],
       [1865, 1861, 1789, 1844, 1955, 1887, 1922, 1918, 1952, 1883, 1938,
        2483, 2429, 2026, 1943, 1944]])